Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
We create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [3]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir


path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [5]:

def train_happy_sad_model():
    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc') > DESIRED_ACCURACY):
                print('\nReached 99.9% accuracy so cancelling training!')
                self.model.stop_training = True
        

    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        # THe first convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(
        loss='binary_crossentropy', 
        optimizer=RMSprop(lr=0.001),
        metrics=['acc']
    )
        
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
    train_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        target_size=(150,150),
        batch_size=10,
        class_mode='binary'
    )
    
    # model fitting
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=8,
        epochs=15,
        verbose=1,
        callbacks=[callbacks]    
    )
    # model fitting
    return history.history['acc'][-1]

In [6]:
train_happy_sad_model()

W0701 17:03:47.685696 139885527476032 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0701 17:03:48.019266 139885527476032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 6s 709ms/step - loss: 4.5957 - acc: 0.5375
Epoch 2/15
8/8 [==============================] - 1s 64ms/step - loss: 0.5337 - acc: 0.6250
Epoch 3/15
8/8 [==============================] - 1s 74ms/step - loss: 0.2260 - acc: 0.9500
Epoch 4/15
8/8 [==============================] - 1s 63ms/step - loss: 0.1737 - acc: 0.9500
Epoch 5/15
8/8 [==============================] - 1s 73ms/step - loss: 0.2019 - acc: 0.9250
Epoch 6/15
8/8 [==============================] - 1s 76ms/step - loss: 0.4900 - acc: 0.8250
Epoch 7/15
8/8 [==============================] - 1s 74ms/step - loss: 0.0629 - acc: 0.9875
Epoch 8/15
6/8 [=====================>........] - ETA: 0s - loss: 0.0406 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 1s 74ms/step - loss: 0.0451 - acc: 1.0000


1.0